In [1]:
import numpy as np
import pandas as pd

In [ ]:
# Create random assignment matrix
array = np.random.randint(0, 2, size=(len(sect_n), len(tas_n)))

In [104]:
import numpy as np

def flip_random_indices(array, x_amount):
    # Generate x_amount unique random indices
    indices = np.random.choice(array.size, x_amount, replace=False)
    array[indices] = 1 - array[indices]
    return array

# Example usage
array = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 1]])
x_amount = 1  # Number of entries to flip

flipped_array = flip_random_indices(array, x_amount)
print("Original Array:\n", array)
print("Flipped Array:\n", flipped_array)


Original Array:
 [[0 1 0]
 [1 0 1]
 [0 1 1]]
Flipped Array:
 [[0 1 1]
 [1 0 1]
 [0 1 1]]


In [109]:
sol = np.array([[0,0],[0,0]])
num_changes = 4
a = np.random.choice(sol.size, num_changes, replace=False)
a

array([3, 0, 2, 1])